In [1]:
# pip install ortools

import numpy as np
from typing import List, Tuple, Dict
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model
import matplotlib.pyplot as plt
from dataclasses import dataclass


@dataclass
class Solution:
    makespan: int
    tardiness: int
    assignments: Dict
    status: str


class MIP_JSSP_MultiObjective:
    def __init__(self, jobs_data, due_dates):
        """
        jobs_data: List of jobs, each job is list of (machine, duration) tuples
        due_dates: List of due dates for each job
        """
        self.jobs_data = jobs_data
        self.due_dates = due_dates
        self.J = len(jobs_data)  # number of jobs
        self.M = max(max(m for m, _ in job) for job in jobs_data) + 1  # number of machines
        self.operations = self._build_operations()
        
    def _build_operations(self):
        """Build flat list of operations with indices"""
        ops = []
        for j, job in enumerate(self.jobs_data):
            for o, (machine, duration) in enumerate(job):
                ops.append({
                    'job': j,
                    'op_in_job': o,
                    'machine': machine,
                    'duration': duration,
                    'op_id': len(ops)
                })
        return ops
    
    def solve_epsilon_constraint(self, max_makespan: int, time_limit: float = 30.0) -> Solution:
        """
        Solve using ε-constraint method: minimize tardiness subject to makespan ≤ max_makespan
        """
        solver = pywraplp.Solver.CreateSolver('SCIP')
        if not solver:
            return Solution(0, 0, {}, "NO_SOLVER")
        
        # Upper bound for timing variables
        H = sum(op['duration'] for op in self.operations) * 2
        
        # Decision variables: start time of each operation
        start_vars = {}
        for op in self.operations:
            start_vars[op['op_id']] = solver.IntVar(0, H, f"start_{op['job']}_{op['op_in_job']}")
        
        # Job completion times
        job_completion = {}
        for j in range(self.J):
            job_completion[j] = solver.IntVar(0, H, f"completion_{j}")
        
        # Makespan variable
        makespan = solver.IntVar(0, H, "makespan")
        
        # Tardiness variables
        tardiness_vars = {}
        for j in range(self.J):
            tardiness_vars[j] = solver.IntVar(0, H, f"tardiness_{j}")
        
        # Total tardiness
        total_tardiness = solver.IntVar(0, H * self.J, "total_tardiness")
        
        # CONSTRAINTS
        
        # 1. Job precedence constraints
        for j in range(self.J):
            job_ops = [op for op in self.operations if op['job'] == j]
            job_ops.sort(key=lambda x: x['op_in_job'])
            
            for i in range(len(job_ops) - 1):
                curr_op = job_ops[i]
                next_op = job_ops[i + 1]
                # Current operation must finish before next starts
                solver.Add(start_vars[curr_op['op_id']] + curr_op['duration'] <= 
                          start_vars[next_op['op_id']])
            
            # Job completion time = end of last operation
            if job_ops:
                last_op = job_ops[-1]
                solver.Add(job_completion[j] == 
                          start_vars[last_op['op_id']] + last_op['duration'])
        
        # 2. Machine capacity constraints (no overlap)
        for m in range(self.M):
            machine_ops = [op for op in self.operations if op['machine'] == m]
            
            # For each pair of operations on the same machine
            for i, op1 in enumerate(machine_ops):
                for j, op2 in enumerate(machine_ops):
                    if i < j:  # avoid duplicate constraints
                        # Binary variable: does op1 precede op2?
                        precedence = solver.BoolVar(f"prec_{op1['op_id']}_{op2['op_id']}")
                        
                        # If op1 precedes op2: start1 + dur1 <= start2
                        M_big = H
                        solver.Add(start_vars[op1['op_id']] + op1['duration'] <= 
                                  start_vars[op2['op_id']] + M_big * (1 - precedence))
                        
                        # If op2 precedes op1: start2 + dur2 <= start1
                        solver.Add(start_vars[op2['op_id']] + op2['duration'] <= 
                                  start_vars[op1['op_id']] + M_big * precedence)
        
        # 3. Makespan constraints
        for j in range(self.J):
            solver.Add(makespan >= job_completion[j])
        
        # 4. Tardiness constraints
        for j in range(self.J):
            solver.Add(tardiness_vars[j] >= job_completion[j] - self.due_dates[j])
            solver.Add(tardiness_vars[j] >= 0)
        
        solver.Add(total_tardiness == sum(tardiness_vars.values()))
        
        # 5. ε-constraint: makespan ≤ max_makespan
        solver.Add(makespan <= max_makespan)
        
        # OBJECTIVE: Minimize total tardiness
        solver.Minimize(total_tardiness)
        
        # Solve
        solver.SetTimeLimit(int(time_limit * 1000))  # milliseconds
        status = solver.Solve()
        
        if status in [pywraplp.Solver.OPTIMAL, pywraplp.Solver.FEASIBLE]:
            # Extract solution
            assignments = {}
            for op in self.operations:
                assignments[op['op_id']] = {
                    'start': int(start_vars[op['op_id']].solution_value()),
                    'job': op['job'],
                    'machine': op['machine'],
                    'duration': op['duration']
                }
            
            return Solution(
                makespan=int(makespan.solution_value()),
                tardiness=int(total_tardiness.solution_value()),
                assignments=assignments,
                status="OPTIMAL" if status == pywraplp.Solver.OPTIMAL else "FEASIBLE"
            )
        else:
            return Solution(0, 0, {}, "INFEASIBLE")
    
    def solve_weighted_sum(self, alpha: float, time_limit: float = 30.0) -> Solution:
        """
        Solve using weighted sum: minimize α*makespan + (1-α)*tardiness
        """
        solver = pywraplp.Solver.CreateSolver('SCIP')
        if not solver:
            return Solution(0, 0, {}, "NO_SOLVER")
        
        # Similar setup as epsilon constraint, but different objective
        H = sum(op['duration'] for op in self.operations) * 2
        
        # Decision variables
        start_vars = {}
        for op in self.operations:
            start_vars[op['op_id']] = solver.IntVar(0, H, f"start_{op['job']}_{op['op_in_job']}")
        
        job_completion = {}
        for j in range(self.J):
            job_completion[j] = solver.IntVar(0, H, f"completion_{j}")
        
        makespan = solver.IntVar(0, H, "makespan")
        
        tardiness_vars = {}
        for j in range(self.J):
            tardiness_vars[j] = solver.IntVar(0, H, f"tardiness_{j}")
        
        total_tardiness = solver.IntVar(0, H * self.J, "total_tardiness")
        
        # Add all constraints (same as epsilon method)
        self._add_constraints(solver, start_vars, job_completion, makespan, tardiness_vars, total_tardiness)
        
        # OBJECTIVE: Weighted sum
        # Scale tardiness to be comparable with makespan
        tardiness_scale = 1.0 / self.J if self.J > 0 else 1.0
        
        # Create weighted objective
        weighted_obj = solver.IntVar(0, H * 10, "weighted_obj")
        solver.Add(weighted_obj == 
                  int(alpha * 100) * makespan + 
                  int((1 - alpha) * 100 * tardiness_scale) * total_tardiness)
        
        solver.Minimize(weighted_obj)
        
        # Solve
        solver.SetTimeLimit(int(time_limit * 1000))
        status = solver.Solve()
        
        if status in [pywraplp.Solver.OPTIMAL, pywraplp.Solver.FEASIBLE]:
            assignments = {}
            for op in self.operations:
                assignments[op['op_id']] = {
                    'start': int(start_vars[op['op_id']].solution_value()),
                    'job': op['job'],
                    'machine': op['machine'],
                    'duration': op['duration']
                }
            
            return Solution(
                makespan=int(makespan.solution_value()),
                tardiness=int(total_tardiness.solution_value()),
                assignments=assignments,
                status="OPTIMAL" if status == pywraplp.Solver.OPTIMAL else "FEASIBLE"
            )
        else:
            return Solution(0, 0, {}, "INFEASIBLE")
    
    def _add_constraints(self, solver, start_vars, job_completion, makespan, tardiness_vars, total_tardiness):
        """Helper method to add all constraints"""
        H = sum(op['duration'] for op in self.operations) * 2
        
        # Job precedence
        for j in range(self.J):
            job_ops = [op for op in self.operations if op['job'] == j]
            job_ops.sort(key=lambda x: x['op_in_job'])
            
            for i in range(len(job_ops) - 1):
                curr_op = job_ops[i]
                next_op = job_ops[i + 1]
                solver.Add(start_vars[curr_op['op_id']] + curr_op['duration'] <= 
                          start_vars[next_op['op_id']])
            
            if job_ops:
                last_op = job_ops[-1]
                solver.Add(job_completion[j] == 
                          start_vars[last_op['op_id']] + last_op['duration'])
        
        # Machine capacity
        for m in range(self.M):
            machine_ops = [op for op in self.operations if op['machine'] == m]
            
            for i, op1 in enumerate(machine_ops):
                for j, op2 in enumerate(machine_ops):
                    if i < j:
                        precedence = solver.BoolVar(f"prec_{op1['op_id']}_{op2['op_id']}")
                        
                        M_big = H
                        solver.Add(start_vars[op1['op_id']] + op1['duration'] <= 
                                  start_vars[op2['op_id']] + M_big * (1 - precedence))
                        solver.Add(start_vars[op2['op_id']] + op2['duration'] <= 
                                  start_vars[op1['op_id']] + M_big * precedence)
        
        # Makespan
        for j in range(self.J):
            solver.Add(makespan >= job_completion[j])
        
        # Tardiness
        for j in range(self.J):
            solver.Add(tardiness_vars[j] >= job_completion[j] - self.due_dates[j])
            solver.Add(tardiness_vars[j] >= 0)
        
        solver.Add(total_tardiness == sum(tardiness_vars.values()))
    
    def generate_pareto_front_epsilon_constraint(self, n_points: int = 10, time_limit: float = 30.0) -> List[Solution]:
        """Generate Pareto front using ε-constraint method"""
        solutions = []
        
        # First, find makespan range by solving single objectives
        print("Finding makespan bounds...")
        
        # Minimize makespan only
        min_makespan_sol = self.solve_weighted_sum(alpha=1.0, time_limit=time_limit)
        if min_makespan_sol.status == "INFEASIBLE":
            return solutions
        
        # Minimize tardiness only  
        min_tardiness_sol = self.solve_weighted_sum(alpha=0.0, time_limit=time_limit)
        if min_tardiness_sol.status == "INFEASIBLE":
            return solutions
        
        min_makespan = min_makespan_sol.makespan
        max_makespan = min_tardiness_sol.makespan
        
        print(f"Makespan range: {min_makespan} - {max_makespan}")
        
        # Generate points along the makespan range
        makespan_values = np.linspace(min_makespan, max_makespan, n_points, dtype=int)
        
        for i, max_ms in enumerate(makespan_values):
            print(f"Solving ε-constraint {i+1}/{n_points} with makespan ≤ {max_ms}")
            sol = self.solve_epsilon_constraint(max_ms, time_limit)
            if sol.status != "INFEASIBLE":
                solutions.append(sol)
        
        return solutions
    
    def generate_pareto_front_weighted_sum(self, n_points: int = 10, time_limit: float = 30.0) -> List[Solution]:
        """Generate Pareto front using weighted sum method"""
        solutions = []
        
        # Generate different weight combinations
        alphas = np.linspace(0, 1, n_points)
        
        for i, alpha in enumerate(alphas):
            print(f"Solving weighted sum {i+1}/{n_points} with α={alpha:.2f}")
            sol = self.solve_weighted_sum(alpha, time_limit)
            if sol.status != "INFEASIBLE":
                solutions.append(sol)
        
        return solutions


# Example usage and comparison
def create_test_problem():
    """Create the same test problem as in the original code"""
    rng = np.random.default_rng(42)
    J, M = 6, 4  # Smaller for MIP tractability
    
    routes = []
    for j in range(J):
        machines = list(rng.permutation(M))
        durations = list(rng.integers(2, 10, size=M))
        job_ops = [(machines[k], int(durations[k])) for k in range(M)]
        routes.append(job_ops)
    
    job_total = [sum(d for _, d in routes[j]) for j in range(J)]
    due_dates = [int(1.5 * job_total[j]) for j in range(J)]
    
    return routes, due_dates


def compare_methods():
    """Compare different multi-objective approaches"""
    print("Creating test problem...")
    jobs_data, due_dates = create_test_problem()
    
    mip_solver = MIP_JSSP_MultiObjective(jobs_data, due_dates)
    
    print("\n" + "="*50)
    print("METHOD 1: ε-Constraint")
    print("="*50)
    
    epsilon_solutions = mip_solver.generate_pareto_front_epsilon_constraint(
        n_points=8, time_limit=15.0
    )
    
    print(f"\nFound {len(epsilon_solutions)} solutions:")
    for i, sol in enumerate(epsilon_solutions):
        print(f"  {i+1}: makespan={sol.makespan}, tardiness={sol.tardiness} ({sol.status})")
    
    print("\n" + "="*50)
    print("METHOD 2: Weighted Sum")
    print("="*50)
    
    weighted_solutions = mip_solver.generate_pareto_front_weighted_sum(
        n_points=8, time_limit=15.0
    )
    
    print(f"\nFound {len(weighted_solutions)} solutions:")
    for i, sol in enumerate(weighted_solutions):
        print(f"  {i+1}: makespan={sol.makespan}, tardiness={sol.tardiness} ({sol.status})")
    
    # Plot comparison
    if epsilon_solutions and weighted_solutions:
        plt.figure(figsize=(10, 6))
        
        # Plot ε-constraint solutions
        eps_makespan = [sol.makespan for sol in epsilon_solutions]
        eps_tardiness = [sol.tardiness for sol in epsilon_solutions]
        plt.scatter(eps_makespan, eps_tardiness, color='blue', label='ε-constraint', s=60, alpha=0.7)
        
        # Plot weighted sum solutions
        ws_makespan = [sol.makespan for sol in weighted_solutions]
        ws_tardiness = [sol.tardiness for sol in weighted_solutions]
        plt.scatter(ws_makespan, ws_tardiness, color='red', label='Weighted sum', s=60, alpha=0.7)
        
        plt.xlabel('Makespan')
        plt.ylabel('Total Tardiness')
        plt.title('Pareto Front Comparison: MIP Methods')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()


if __name__ == "__main__":
    compare_methods()

Creating test problem...

METHOD 1: ε-Constraint
Finding makespan bounds...

Found 0 solutions:

METHOD 2: Weighted Sum
Solving weighted sum 1/8 with α=0.00
Solving weighted sum 2/8 with α=0.14
Solving weighted sum 3/8 with α=0.29
Solving weighted sum 4/8 with α=0.43
Solving weighted sum 5/8 with α=0.57
Solving weighted sum 6/8 with α=0.71
Solving weighted sum 7/8 with α=0.86
Solving weighted sum 8/8 with α=1.00

Found 5 solutions:
  1: makespan=280, tardiness=8 (OPTIMAL)
  2: makespan=45, tardiness=8 (OPTIMAL)
  3: makespan=45, tardiness=8 (OPTIMAL)
  4: makespan=42, tardiness=22 (OPTIMAL)
  5: makespan=42, tardiness=22 (OPTIMAL)


Above code is MIP-Based Multi-Objective Optimization
Replace NSGA-II with MIP using ε-constraint or weighted sum methods: